In [25]:
import pandas as pd
import numpy as np
import yfinance as yf

def carregar_dados(ticker, data_inicio, data_fim):
    """
    Busca dados históricos e adiciona a última cotação intraday, se for de um novo dia.
    """
    # 1. Cria o objeto Ticker para o ativo desejado
    require = yf.Ticker(ticker)
    name_ticker = require.info.get('longName', 'Unknown')

    # 2. Busca o histórico diário principal
    data = require.history(start=data_inicio, end=data_fim)

    # 3. Busca dados intraday bem recentes (últimos 2 dias, intervalo de 1 minuto)
    bd = require.history(period="2d", interval='1m')

    # 4. Compara as datas e concatena se necessário
    # Checa se o último dia do histórico é diferente do último dia do intraday
    if data.index[-1].date() != bd.index[-1].date():
        # Se forem diferentes, anexa a última cotação intraday ao histórico
        # Usamos pd.concat para garantir que os dataframes sejam unidos corretamente
        data = pd.concat([data, bd.tail(1)])
        
    return data, name_ticker

In [28]:
bd, name_ticker = carregar_dados('BOVA11.SA', '2016-01-01', '2026-02-09')
print(name_ticker)
print(bd.tail())

iShares Ibovespa Index Fund
                                 Open        High         Low       Close  \
2026-02-03 00:00:00-03:00  181.660004  183.809998  180.789993  182.190002   
2026-02-04 00:00:00-03:00  181.160004  181.279999  176.759995  178.199997   
2026-02-05 00:00:00-03:00  178.550003  180.669998  178.020004  178.649994   
2026-02-06 00:00:00-03:00  178.940002  179.800003  177.910004  179.240005   
2026-02-13 16:59:00-03:00  182.419998  182.419998  182.320007  182.419998   

                            Volume  Dividends  Stock Splits  
2026-02-03 00:00:00-03:00  5835580        0.0           0.0  
2026-02-04 00:00:00-03:00  7115202        0.0           0.0  
2026-02-05 00:00:00-03:00  5358557        0.0           0.0  
2026-02-06 00:00:00-03:00  2905591        0.0           0.0  
2026-02-13 16:59:00-03:00     8461        0.0           0.0  


In [35]:
from bizdays import Calendar

dates = pd.to_datetime(bd.index)
last_date = list(dates)[-1]
# Configuração única
cal = Calendar.load("B3")
# Gera os próximos 10 dias úteis da B3
# forecast_dates = [pd.to_datetime(cal.offset(last_date, i)) for i in range(1, 10+1)]
forecast_dates = pd.to_datetime(cal.offset(last_date, range(1, 10+1)))

original_tz = bd.index.tz
original_tz

if original_tz is not None:
    forecast_dates = forecast_dates.tz_localize(original_tz)
forecast_dates

DatetimeIndex(['2026-02-18 00:00:00-03:00', '2026-02-19 00:00:00-03:00',
               '2026-02-20 00:00:00-03:00', '2026-02-23 00:00:00-03:00',
               '2026-02-24 00:00:00-03:00', '2026-02-25 00:00:00-03:00',
               '2026-02-26 00:00:00-03:00', '2026-02-27 00:00:00-03:00',
               '2026-03-02 00:00:00-03:00', '2026-03-03 00:00:00-03:00'],
              dtype='datetime64[ns, America/Sao_Paulo]', freq=None)